In [7]:
import requests
import pandas as pd
import re
import numpy as np

def name_standardization(colname,df):
    df[colname] = df[colname].str.lower().str.replace('.','',regex=False)
    df[colname] = df[colname].str.replace('\'','',regex=True).str.replace(r'\s+(ii|iii|iv|v|vi|vii|viii|ix|x)$', '', case=False, regex=True)
    df.rename(columns={'game.week':'week'},inplace=True)
    return df


df = pd.read_csv('raw_data/wr_2024_non_rookies.csv')

#df.sort_values('Start  Year',ascending=False)
df.columns = [
    re.sub(r'\s+', ' ', col)     # collapse multiple spaces into one
      .strip()                   # remove leading/trailing spaces
      .replace(" ", "_")         # replace spaces with underscores
      .replace("%", "percent")   # replace % with 'percent'
      .replace("_percent_percent","_percent")
      .replace("-","")
      .lower()                   # make everything lowercase
    for col in df.columns
]

salary_cols = [
    "value",
    "average_salary",
    "signing_bonus",
    "guarantee_at_sign",
    "practical_guarantee",
    "2year_cash_total",
    "3year_cash_total"
]

for col in salary_cols:
    df[col] = (
        df[col]
        .replace(r"[\$,]", "", regex=True)   # remove $ and commas
        .astype(float)                       # convert to numeric
    )
df = name_standardization("player",df)
df = df.loc[df.age_at_signing>23] #remove rookies


conditions = [
    df["average_salary"] >= 30_000_000,
    (df["average_salary"] >= 17_000_000) & (df["average_salary"] < 30_000_000),
    (df["average_salary"] >= 5_000_000) & (df["average_salary"] < 17_000_000),
    df["average_salary"] < 5_000_000
]

choices = ["tier1", "tier2", "tier3", "tier4"]

df["tier"] = np.select(conditions, choices, default="tier4")

df.sort_values('value',ascending=False)

,rk,player,pos,team_currently_with,age_at_signing,start_year,end_year,yrs,value,average_salary,average_percent_of_cap_at_sign,signing_bonus,guarantee_at_sign,practical_guarantee,2year_cash_total,3year_cash_total,tier
0,1,justin jefferson,WR,MIN MIN,25,2024,2028,4,140000000.0,35000000.0,13.70%,36938000.0,88743000.0,110000000.0,70243000.0,95743000.0,tier1
1,2,ceedee lamb,WR,DAL DAL,25,2024,2029,4,136000000.0,34000000.0,13.31%,38000000.0,67000000.0,100000000.0,67007560.0,93007560.0,tier1
2,3,amon-ra st brown,WR,DET DET,24,2024,2028,4,120010000.0,30002500.0,11.75%,16500000.0,34666000.0,77000000.0,35276000.0,63386000.0,tier1
3,4,brandon aiyuk,WR,SF SF,26,2024,2028,4,120000000.0,30000000.0,11.75%,23000000.0,45000000.0,76000000.0,48906300.0,75906300.0,tier1
4,5,dj moore,WR,CHI CHI,27,2024,2029,4,110000000.0,27500000.0,10.77%,20000000.0,43650000.0,81525000.0,44281460.0,68781460.0,tier2
5,6,aj brown,WR,PHI PHI,26,2024,2029,3,96000000.0,32000000.0,12.53%,19875000.0,51000000.0,84000000.0,51000000.0,80000000.0,tier1
6,7,calvin ridley,WR,TEN TEN,29,2024,2027,4,92000000.0,23000000.0,9.01%,20000000.0,46980000.0,50000000.0,48007560.0,69757560.0,tier2
7,8,tyreek hill,WR,MIA MIA,30,2024,2026,3,90000000.0,30000000.0,11.75%,7000000.0,54000000.0,59000000.0,54100000.0,90100000.0,tier1
8,9,jaylen waddle,WR,MIA MIA,25,2024,2028,3,84750000.0,28250000.0,11.06%,18873000.0,35978000.0,76000000.0,36082834.0,53323834.0,tier2
9,10,devonta smith,WR,PHI PHI,25,2024,2028,3,75000000.0,25000000.0,9.79%,20351000.0,33997554.0,51000000.0,34000074.0,48000074.0,tier2


In [10]:
import pandas as pd

data = [
    ('justin jefferson', '6/24/2024'),
    ('ceedee lamb', '8/26/2024'),
    ('amon-ra st brown', '4/15/2024'),
    ('brandon aiyuk', '8/15/2024'),
    ('dj moore', '7/31/2024'),
    ('aj brown', '4/15/2024'),
    ('calvin ridley', '3/15/2024'),
    ('tyreek hill', '8/3/2024'),
    ('jaylen waddle', '5/30/2024'),
    ('devonta smith', '4/15/2024'),
    ('nico collins', '5/15/2024'),
    ('michael pittman jr', '3/11/2024'),
    ('jerry jeudy', '3/18/2024'),
    ('mike evans', '3/4/2024'),
    ('curtis samuel', '3/14/2024'),
    ('jauan jennings', '5/29/2024'),
    ('darnell mooney', '3/12/2024'),
    ('rashid shaheed','7/20/2024'),
    ('ray-ray mccloud','3/18/2024'),
    ('brandon powell','3/13/2024'),
    ('deonte harty','4/10/2024'),
    ('nsimba webster','1/8/2024')
]

contract_date = pd.DataFrame(data, columns=['player', 'contract_date'])

# convert contract_date to datetime
contract_date['contract_date'] = pd.to_datetime(contract_date['contract_date'], format="%m/%d/%Y")


df = df.loc[df['start_year']==2024]
df2 = pd.merge(df,contract_date, left_on = 'player', right_on = 'player', how='left')#df.sort_values('contract_date')
df2.sort_values('contract_date')

,rk,player,pos,team_currently_with,age_at_signing,start_year,end_year,yrs,value,average_salary,average_percent_of_cap_at_sign,signing_bonus,guarantee_at_sign,practical_guarantee,2year_cash_total,3year_cash_total,tier,contract_date
20,24,nsimba webster,WR,CHI CHI,27,2024,2024,1,1055000.0,1055000.0,0.41%,NaN,NaN,NaN,545000.0,545000.0,tier4,2024-01-08
12,13,mike evans,WR,TB TB,30,2024,2025,2,41000000.0,20500000.0,8.03%,NaN,29000000.0,35000000.0,44666668.0,44666668.0,tier2,2024-03-04
13,14,darnell mooney,WR,ATL ATL,26,2024,2026,3,39000000.0,13000000.0,5.09%,15000000.0,26000000.0,26000000.0,27171276.0,39151276.0,tier3,2024-03-12
18,22,brandon powell,WR,MIN MIN,27,2024,2024,1,2000000.0,2000000.0,0.78%,375000.0,625000.0,625000.0,1977941.0,1977941.0,tier4,2024-03-13
14,15,curtis samuel,WR,BUF BUF,27,2024,2026,3,24000000.0,8000000.0,3.13%,6900000.0,14020000.0,15020000.0,15988441.0,23988441.0,tier3,2024-03-14
6,7,calvin ridley,WR,TEN TEN,29,2024,2027,4,92000000.0,23000000.0,9.01%,20000000.0,46980000.0,50000000.0,48007560.0,69757560.0,tier2,2024-03-15
17,18,ray-ray mccloud,WR,ATL ATL,27,2024,2025,2,5000000.0,2500000.0,0.98%,1000000.0,2125000.0,2125000.0,6723580.0,6723580.0,tier4,2024-03-18
11,12,jerry jeudy,WR,CLE CLE,24,2024,2027,3,52500000.0,17500000.0,6.85%,6013000.0,41000000.0,41000000.0,35241844.0,48728844.0,tier2,2024-03-18
19,23,deonte harty,WR,BAL BAL,26,2024,2024,1,1292500.0,1292500.0,0.51%,167500.0,1152500.0,1152500.0,1313439.0,1313439.0,tier4,2024-04-10
2,3,amon-ra st brown,WR,DET DET,24,2024,2028,4,120010000.0,30002500.0,11.75%,16500000.0,34666000.0,77000000.0,35276000.0,63386000.0,tier1,2024-04-15


In [17]:

stats = pd.read_csv('raw_data/nfl_training_data_2009_2024_v2.csv')
stats.loc[(stats.pos=='WR')&(stats.year >= 2022)]

stats.loc[(stats.pos=='WR') & (stats.player.str.contains('jefferson'))]

,player,year,pos,college,dob,age_season_start,height,weight,years_exp,team,...,rush_yds_per_game,fpts_standard,fpts_half_ppr,fpts_ppr,fpts_ppr_f1,adp,adp_f1,depth,depth_f1,is_majorly_injured
4592,justin jefferson,2020,WR,LSU,1999-06-16,21.227397,73,195,0,['MIN'],...,0.1,181.2,225.2,269.2,330.0,125.5,23.1,1.0,1.0,0
4593,justin jefferson,2021,WR,LSU,1999-06-16,22.227397,73,195,1,['MIN'],...,0.8,222.0,276.0,330.0,365.3,23.1,5.4,1.0,1.0,0
4594,justin jefferson,2022,WR,LSU,1999-06-16,23.227397,73,195,2,['MIN'],...,1.4,237.3,301.3,365.3,203.2,5.4,2.0,1.0,1.0,0
4595,justin jefferson,2023,WR,LSU,1999-06-16,24.227397,73,195,3,['MIN'],...,-1.2,135.2,169.2,203.2,315.6,2.0,4.0,1.0,1.0,0
4596,justin jefferson,2024,WR,LSU,1999-06-16,25.230137,73,195,4,['MIN'],...,0.2,212.6,264.1,315.6,0.0,4.0,9999.0,1.0,3.0,0
7974,van jefferson,2020,WR,"Mississippi,Florida",1996-07-26,24.117808,73,200,0,['LAR'],...,-0.1,26.9,36.4,45.9,168.2,288.0,183.0,2.0,1.0,0
7975,van jefferson,2021,WR,"Mississippi,Florida",1996-07-26,25.117808,73,200,1,['LAR'],...,1.2,118.2,143.2,168.2,78.9,183.0,179.0,1.0,1.0,0
7976,van jefferson,2022,WR,"Mississippi,Florida",1996-07-26,26.117808,73,200,2,['LAR'],...,0.0,54.9,66.9,78.9,41.3,179.0,221.0,1.0,1.0,0
7977,van jefferson,2023,WR,"Mississippi,Florida",1996-07-26,27.117808,73,200,3,"['ATL', 'LAR']",...,0.2,21.3,31.3,41.3,63.6,221.0,326.0,1.0,1.0,0
7978,van jefferson,2024,WR,"Mississippi,Florida",1996-07-26,28.120548,73,200,4,['PIT'],...,0.0,39.6,51.6,63.6,0.0,326.0,9999.0,1.0,3.0,0


In [18]:
wr = stats.loc[(stats.year>=2022)&(stats.pos=='WR')]
wr["stats_rank"] = (
    wr.groupby(["year", "pos"])["fpts_standard"]
      .rank(method="dense", ascending=False)
)
wr

/var/folders/1q/q70ps9k517s2pvt8ycs64xth0000gn/T/ipykernel_63375/266912382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wr["stats_rank"] = (


,player,year,pos,college,dob,age_season_start,height,weight,years_exp,team,...,fpts_standard,fpts_half_ppr,fpts_ppr,fpts_ppr_f1,adp,adp_f1,depth,depth_f1,is_majorly_injured,stats_rank
54,adam thielen,2022,WR,Minnesota St.,1990-08-22,32.049315,74,195,8,['MIN'],...,108.0,143.0,178.0,227.0,57.4,136.0,1.0,1.0,0,32.0
55,adam thielen,2023,WR,Minnesota St.,1990-08-22,33.049315,74,195,9,['CAR'],...,124.0,175.5,227.0,139.5,136.0,158.0,1.0,1.0,0,31.0
56,adam thielen,2024,WR,Minnesota St.,1990-08-22,34.052055,74,195,10,['CAR'],...,91.5,115.5,139.5,0.0,158.0,9999.0,1.0,3.0,0,44.0
62,adonai mitchell,2024,WR,"Georgia,Texas",2002-10-08,21.915068,74,205,0,['IND'],...,29.8,41.3,52.8,0.0,137.0,9999.0,2.0,3.0,0,100.0
94,ainias smith,2024,WR,Texas A&M,2001-05-31,23.271233,69,176,0,['PHI'],...,10.7,14.2,17.7,0.0,581.0,9999.0,3.0,3.0,0,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8217,zay flowers,2023,WR,Boston Col.,2000-09-11,22.986301,69,175,0,['BAL'],...,127.4,165.9,204.4,209.5,105.0,60.0,1.0,1.0,0,28.0
8218,zay flowers,2024,WR,Boston Col.,2000-09-11,23.989041,69,175,1,['BAL'],...,135.5,172.5,209.5,0.0,60.0,9999.0,1.0,3.0,0,22.0
8224,zay jones,2022,WR,East Carolina,1995-03-30,27.443836,74,200,5,['JAX'],...,114.1,155.1,196.1,78.1,191.0,154.0,1.0,1.0,0,27.0
8225,zay jones,2023,WR,East Carolina,1995-03-30,28.443836,74,200,6,['JAX'],...,44.1,61.1,78.1,16.4,154.0,424.0,1.0,3.0,0,83.0


In [20]:
wr = wr[['player', 'year', 'pos', 'college', 'dob', 'age_season_start', 'height',
       'weight', 'years_exp', 'team', 'games_played',
       'games_started','rec_tgt', 'rec', 'rec_yds', 'rec_td','fpts_standard','stats_rank']]

df3 = pd.merge(df2['player'],wr,on='player',how='inner')
df3.to_csv('wr_stats_for_ai_negotiator.csv',index=False)
df3

,player,year,pos,college,dob,age_season_start,height,weight,years_exp,team,games_played,games_started,rec_tgt,rec,rec_yds,rec_td,fpts_standard,stats_rank
0,justin jefferson,2022,WR,LSU,1999-06-16,23.227397,73,195,2,['MIN'],17,17,184,128,1809,8,237.3,1.0
1,justin jefferson,2023,WR,LSU,1999-06-16,24.227397,73,195,3,['MIN'],10,9,100,68,1074,5,135.2,24.0
2,justin jefferson,2024,WR,LSU,1999-06-16,25.230137,73,195,4,['MIN'],17,17,154,103,1533,10,212.6,1.0
3,ceedee lamb,2022,WR,Oklahoma,1999-04-08,23.416438,74,200,2,['DAL'],17,17,156,107,1359,9,194.6,6.0
4,ceedee lamb,2023,WR,Oklahoma,1999-04-08,24.416438,74,200,3,['DAL'],17,17,181,135,1749,12,267.2,1.0
5,ceedee lamb,2024,WR,Oklahoma,1999-04-08,25.419178,74,200,4,['DAL'],15,15,152,101,1194,6,161.4,7.0
6,amon-ra st brown,2022,WR,USC,1999-10-24,22.871233,72,202,1,['DET'],16,16,146,106,1161,6,161.6,9.0
7,amon-ra st brown,2023,WR,USC,1999-10-24,23.871233,72,202,2,['DET'],16,16,164,119,1515,10,212.9,3.0
8,amon-ra st brown,2024,WR,USC,1999-10-24,24.873973,72,202,3,['DET'],17,17,141,115,1263,12,197.9,2.0
9,brandon aiyuk,2022,WR,Arizona St.,1998-03-17,24.476712,72,200,2,['SF'],17,17,114,78,1015,8,150.8,14.0


In [21]:

# Drop the "Player" column
df4 = df2.drop(columns=["rk"])

# Move 'player' column to the front
cols = ["player"] + [c for c in df4.columns if c != "player"]
df4 = df4[cols]


df4.to_csv('contract_details.csv',index=False)
df4

,player,pos,team_currently_with,age_at_signing,start_year,end_year,yrs,value,average_salary,average_percent_of_cap_at_sign,signing_bonus,guarantee_at_sign,practical_guarantee,2year_cash_total,3year_cash_total,tier,contract_date
0,justin jefferson,WR,MIN MIN,25,2024,2028,4,140000000.0,35000000.0,13.70%,36938000.0,88743000.0,110000000.0,70243000.0,95743000.0,tier1,2024-06-24
1,ceedee lamb,WR,DAL DAL,25,2024,2029,4,136000000.0,34000000.0,13.31%,38000000.0,67000000.0,100000000.0,67007560.0,93007560.0,tier1,2024-08-26
2,amon-ra st brown,WR,DET DET,24,2024,2028,4,120010000.0,30002500.0,11.75%,16500000.0,34666000.0,77000000.0,35276000.0,63386000.0,tier1,2024-04-15
3,brandon aiyuk,WR,SF SF,26,2024,2028,4,120000000.0,30000000.0,11.75%,23000000.0,45000000.0,76000000.0,48906300.0,75906300.0,tier1,2024-08-15
4,dj moore,WR,CHI CHI,27,2024,2029,4,110000000.0,27500000.0,10.77%,20000000.0,43650000.0,81525000.0,44281460.0,68781460.0,tier2,2024-07-31
5,aj brown,WR,PHI PHI,26,2024,2029,3,96000000.0,32000000.0,12.53%,19875000.0,51000000.0,84000000.0,51000000.0,80000000.0,tier1,2024-04-15
6,calvin ridley,WR,TEN TEN,29,2024,2027,4,92000000.0,23000000.0,9.01%,20000000.0,46980000.0,50000000.0,48007560.0,69757560.0,tier2,2024-03-15
7,tyreek hill,WR,MIA MIA,30,2024,2026,3,90000000.0,30000000.0,11.75%,7000000.0,54000000.0,59000000.0,54100000.0,90100000.0,tier1,2024-08-03
8,jaylen waddle,WR,MIA MIA,25,2024,2028,3,84750000.0,28250000.0,11.06%,18873000.0,35978000.0,76000000.0,36082834.0,53323834.0,tier2,2024-05-30
9,devonta smith,WR,PHI PHI,25,2024,2028,3,75000000.0,25000000.0,9.79%,20351000.0,33997554.0,51000000.0,34000074.0,48000074.0,tier2,2024-04-15


In [96]:
df

,RK,Player,Pos,Team Currently With,Age At Signing,Start Year,End Year,Yrs,Value,Average Salary,Guarantee at Sign,Practical Guarantee
1,2,justin jefferson,WR,MIN MIN,25,2024,2028,4,"$140,000,000","$35,000,000","$88,743,000","$110,000,000"
2,3,ceedee lamb,WR,DAL DAL,25,2024,2029,4,"$136,000,000","$34,000,000","$67,000,000","$100,000,000"
5,6,amon-ra st brown,WR,DET DET,24,2024,2028,4,"$120,010,000","$30,002,500","$34,666,000","$77,000,000"
6,7,brandon aiyuk,WR,SF SF,26,2024,2028,4,"$120,000,000","$30,000,000","$45,000,000","$76,000,000"
8,9,dj moore,WR,CHI CHI,27,2024,2029,4,"$110,000,000","$27,500,000","$43,650,000","$81,525,000"
10,11,aj brown,WR,PHI PHI,26,2024,2029,3,"$96,000,000","$32,000,000","$51,000,000","$84,000,000"
12,T12,calvin ridley,WR,TEN TEN,29,2024,2027,4,"$92,000,000","$23,000,000","$46,980,000","$50,000,000"
13,14,tyreek hill,WR,MIA MIA,30,2024,2026,3,"$90,000,000","$30,000,000","$54,000,000","$59,000,000"
14,15,jaylen waddle,WR,MIA MIA,25,2024,2028,3,"$84,750,000","$28,250,000","$35,978,000","$76,000,000"
16,17,devonta smith,WR,PHI PHI,25,2024,2028,3,"$75,000,000","$25,000,000","$33,997,554","$51,000,000"
